In [0]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


W0626 07:33:02.338815 140425501366144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0626 07:33:02.340164 140425501366144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0626 07:33:02.356683 140425501366144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0626 07:33:02.366372 140425501366144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0626 07:33:02.382000 140425501366144 deprecation_wrapper.py:119] From /usr/local/lib/

In [0]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents":[{"tag":"greeting",\r\n\t\t"patterns":["Hi","How are you?","Is anyone there?","Hello","Good day"],\r\n\t\t"responses":["Hello,thanks,for visiting","Good to see you again","Hi, there , how can i help?"],\r\n\t\t"context_set":""\r\n\t\t},\r\n\t\t{"tag":"goodbye",\r\n\t\t"patterns":["bye","See you later","Good bye","Good day"],\r\n\t\t"responses":["See you later, thanks for visiting","Have a nice day","Bye! Come back again soon"]\r\n\t\t},\r\n\t\t{"tag":"hours",\r\n\t\t"patterns":["what hours are you open?","When are you open"],\r\n\t\t"responses":["We are open every day 9am-9pm"," our hours are 9am-9pm every day"]\r\n\t\t},\r\n\t\t{"tag":"Location",\r\n\t\t"patterns":["what is your location?","Where are you located?","what is your address?"],\r\n\t\t"responses":["We are located at ville parle."," We are situated at ville parle"]\r\n\t\t},\r\n\t\t{"tag": "payments",\r\n         "patterns": ["Do you take credit cards?", "Do you accept Mastercard?", "Are you ca

In [0]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [0]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you?', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello,thanks,for visiting',
    'Good to see you again',
    'Hi, there , how can i help?'],
   'tag': 'greeting'},
  {'patterns': ['bye', 'See you later', 'Good bye', 'Good day'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon'],
   'tag': 'goodbye'},
  {'patterns': ['what hours are you open?', 'When are you open'],
   'responses': ['We are open every day 9am-9pm',
    ' our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['what is your location?',
    'Where are you located?',
    'what is your address?'],
   'responses': ['We are located at ville parle.',
    ' We are situated at ville parle'],
   'tag': 'Location'},
  {'patterns': ['Do you take credit cards?',
    'Do you accept Mastercard?',
    'Are you cash only?'],
   'responses': ['We accept VISA, Mastercard and AMEX',


In [0]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append(( w,intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [0]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
8 classes ['Location', 'deliveryoption', 'goodbye', 'greeting', 'hours', 'menu', 'payments', 'todaysmenu']
51 unique stemmed words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'hello', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'see', 'serv', 'spec', 'tak', 'tel', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [0]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])


In [0]:
# resetting underlying graph data
tf.reset_default_graph()

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3030  | total loss: 0.49754 | time: 0.010s
| Adam | epoch: 758 | loss: 0.49754 - acc: 0.9113 -- iter: 16/31
